In [396]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import seaborn as sns
import os
import spacy
from datetime import datetime, date
nlp = spacy.load('en')
from collections import Counter
%matplotlib inline

list_df = pd.read_csv('./archive/listings.csv')
cal_df = pd.read_csv('./archive/calendar.csv')
review_df = pd.read_csv('./archive/reviews.csv')


In [236]:
num = '85.00'
float(num)

85.0

In [212]:
for i in list_df.index:
    [x.replace('"','') for x in list_df.amenities[i].strip('{}').split(',')]

In [213]:
for i in amen_set:
    for j in list_df.index:
        if i in [x.replace('"','') for x in list_df.amenities[j].strip('{}').split(',')]:
            list_df.amenities[j]

In [214]:
amenities_list = []
for i in range(len(list_df.amenities)):
    amenities_list.append([x.replace('"','') for x in list_df.amenities[i].strip('{}').split(',')])
flat_list = [item for sublist in amenities_list for item in sublist]
amen_set = set(flat_list)

In [215]:
list_df[list_df.id.isin(reviews_list)]

NameError: name 'reviews_list' is not defined

In [216]:
review1 = nlp(list(review_df.loc[review_df['listing_id']==9075558].comments)[0])

In [217]:
list_df.loc[list_df['review_scores_rating']==57.0]

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
1972,9075558,https://www.airbnb.com/rooms/9075558,20160104002432,2016-01-04,Private Bedroom in a Modern House,My brand new townhouse on the third row from s...,NaN,My brand new townhouse on the third row from s...,none,"15-20 minutes to the downtown by E-line, while...",...,7.0,f,NaN,WASHINGTON,f,moderate,f,f,1,2.95


In [218]:
review1

Accommodation was alright, bed was ok and location very close to downtown. Unfortunately host wasn't at home when we arrived so the room was dirty after last guest and bathroom as well. We had to change our bed linen to clean one (not really sure if it was clean). Benefits are location and price but overall impression is below the average.

In [219]:
words = [token.text for token in review1 if not token.is_stop and not token.is_punct]
word_freq = Counter(words)

In [220]:
nouns = []
adjectives = []
for token in review1:
    if token.pos_ == 'NOUN':
        nouns.append(token)
    if token.pos_ == 'PRP':
        adjectives.append(token)

In [221]:
for chunk in review1.noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_,
            chunk.root.head.text)

Accommodation Accommodation nsubj was
bed bed nsubj was
location location conj was
downtown downtown pobj to
host host nsubj was
home home pobj at
we we nsubj arrived
the room room nsubj was
last guest guest pobj after
bathroom bathroom conj guest
We We nsubj had
our bed linen linen dobj change
it it nsubj was
Benefits Benefits nsubj are
location location attr are
price price conj location
overall impression impression nsubj is
the average average pobj below


In [222]:
quantitative = ['host_since', 'host_response_rate', 'host_acceptance_rate',
       'host_listings_count', 'host_total_listings_count', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'square_feet', 'security_deposit',
       'cleaning_fee', 'guests_included', 'extra_people', 'minimum_nights',
       'maximum_nights', 'availability_30', 'availability_60', 'availability_90',
       'availability_365', 'number_of_reviews', 'first_review', 'last_review', 'review_scores_rating',
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value', 'reviews_per_month']

In [223]:
categorical = ['host_location', 'host_response_time','host_is_superhost', 'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'neighbourhood_group_cleansed', 'property_type', 'room_type', 'bed_type', 'amenities', 'instant_bookable',
       'cancellation_policy', 'require_guest_profile_picture',
       'require_guest_phone_verification']

In [359]:
pd.isnull(list_df['first_review'][0])

False

In [384]:
list_df['first_review'][0]

'2011-11-01'

In [389]:
list_df[quantitative]

,host_since,host_response_rate,host_acceptance_rate,host_listings_count,host_total_listings_count,accommodates,bathrooms,bedrooms,beds,square_feet,...,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,3346,96,100%,3.0,3.0,4,1.0,1.0,1.0,NaN,...,2011-11-01,2016-01-02,95.0,10.0,10.0,10.0,10.0,9.0,10.0,4.07
1,2786,98,100%,6.0,6.0,4,1.0,1.0,1.0,NaN,...,2013-08-19,2015-12-29,96.0,10.0,10.0,10.0,10.0,10.0,10.0,1.48
2,2310,67,100%,2.0,2.0,11,4.5,5.0,7.0,NaN,...,2014-07-30,2015-09-03,97.0,10.0,10.0,10.0,10.0,10.0,10.0,1.15
3,2528,NaN,NaN,1.0,1.0,3,1.0,0.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3236,100,NaN,2.0,2.0,6,2.0,3.0,3.0,NaN,...,2012-07-10,2015-10-24,92.0,9.0,9.0,10.0,10.0,9.0,9.0,0.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3813,2005,99,100%,354.0,354.0,6,2.0,3.0,3.0,NaN,...,2015-09-27,2015-09-27,80.0,8.0,10.0,4.0,8.0,10.0,8.0,0.30
3814,1821,100,100%,1.0,1.0,4,1.0,1.0,2.0,NaN,...,2015-12-18,2015-12-24,100.0,10.0,10.0,10.0,10.0,10.0,10.0,2.00
3815,1744,NaN,NaN,1.0,1.0,2,1.0,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3816,2105,100,NaN,1.0,1.0,2,1.0,0.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [250]:
for x in list_df[quantitative].columns:
    print(x, type(list_df[x][0]))

host_since <class 'str'>
host_response_rate <class 'str'>
host_acceptance_rate <class 'str'>
host_listings_count <class 'numpy.float64'>
host_total_listings_count <class 'numpy.float64'>
accommodates <class 'numpy.int64'>
bathrooms <class 'numpy.float64'>
bedrooms <class 'numpy.float64'>
beds <class 'numpy.float64'>
square_feet <class 'numpy.float64'>
security_deposit <class 'float'>
cleaning_fee <class 'float'>
guests_included <class 'numpy.int64'>
extra_people <class 'str'>
minimum_nights <class 'numpy.int64'>
maximum_nights <class 'numpy.int64'>
availability_30 <class 'numpy.int64'>
availability_60 <class 'numpy.int64'>
availability_90 <class 'numpy.int64'>
availability_365 <class 'numpy.int64'>
number_of_reviews <class 'numpy.int64'>
first_review <class 'str'>
last_review <class 'str'>
review_scores_rating <class 'numpy.float64'>
review_scores_accuracy <class 'numpy.float64'>
review_scores_cleanliness <class 'numpy.float64'>
review_scores_checkin <class 'numpy.float64'>
review_scor

In [392]:
float(list_df.extra_people[0].strip('$'))

5.0

In [283]:
date_obj = date.fromisoformat(list_df.first_review[0])

In [284]:
date_obj

datetime.date(2011, 11, 1)

In [270]:
now = date.today()

In [296]:
now

datetime.date(2020, 10, 5)

In [336]:
diff = date.today() - date.fromisoformat(list_df.first_review[0])

In [337]:
diff.days

3261

1601923262.106114

In [355]:
for i in list_df[date_col].index:
    if not np.isnan(list_df[date_col][i]):
        delta = date.today() - date.fromisoformat(list_df[date_col][i])
        list_df[date_col][x] = delta

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [367]:
#Transform date column to column of integer days

date_col = 'host_since'
for i in list_df[date_col].index:
    if not pd.isnull(list_df[date_col][i]):
        delta = date.today() - date.fromisoformat(list_df[date_col][i])
        list_df[date_col][i] = delta.days
    else:
        next

TypeError: fromisoformat: argument must be str

In [381]:
percent_col = 'host_response_rate'
for i in list_df[percent_col].index:
    if not pd.isnull(list_df[percent_col][i]):
        list_df[percent_col][i] = int(list_df[percent_col][i].strip('%'))

/Users/justinharrison/thesis/env/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [395]:
list_df.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary',
       'space', 'description', 'experiences_offered', 'neighborhood_overview',
       'notes', 'transit', 'thumbnail_url', 'medium_url', 'picture_url',
       'xl_picture_url', 'host_id', 'host_url', 'host_name', 'host_since',
       'host_location', 'host_about', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood',
       'host_listings_count', 'host_total_listings_count',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'street', 'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'country_code', 'country', 'latitude', 'longitude',
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', '